In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q kaggle
!mkdir -p .kaggle
!cp "./drive/My Drive/Study/config/kaggle.json" .kaggle/
!chmod 600 .kaggle/kaggle.json
!mv .kaggle /root

#!kaggle datasets download "birdcall-spectrogram-images-cut-multi"
#!unzip birdcall-spectrogram-images-cut-multi.zip > /dev/null
#!rm -rf birdcall-spectrogram-images-cut-multi.zip

#!kaggle datasets download "birdcall-spectrogram-images-cut"
#!unzip birdcall-spectrogram-images-cut.zip > /dev/null
#!rm -rf birdcall-spectrogram-images-cut.zip

!kaggle datasets download "birdcall-spectrogram-images"
!unzip birdcall-spectrogram-images.zip > /dev/null
!rm -rf birdcall-spectrogram-images.zip

#!kaggle datasets download "birdcall-spectrogram-images-from-torch"
#!unzip birdcall-spectrogram-images-from-torch.zip > /dev/null
#!rm -rf birdcall-spectrogram-images-from-torch.zip

from PIL import Image
from pathlib import Path

#for directory in Path("train_img_2").iterdir():
#for directory in Path("train_img_4").iterdir():
for directory in Path("train_img_5").iterdir():
#for directory in Path("train_img_6").iterdir():
    if directory.name == ".DS_Store":
        continue
    file_paths = [f for f in directory.iterdir() if f.name != ".DS_Store"]
    for path in file_paths:
        try:
            with open(path, 'rb') as f: img = Image.open(f)
        except:
            print(path)
            !rm {path}
#!cp -r "./drive/My Drive/Study/Bird/input/cut_image_from_resnet18_12_nocall/0_nocall" "train_img_2/"
#!cp -r "./drive/My Drive/Study/Bird/input/nocall_20200811" "train_img_4/0_nocall"

In [ ]:
#!pip install panns-inference
#!pip install torchlibrosa
#!git clone https://github.com/qiuqiangkong/audioset_tagging_cnn.git
#!touch audioset_tagging_cnn/__init__.py
#!touch audioset_tagging_cnn/pytorch/__init__.py

In [ ]:
#import sys
#sys.path.append("/content/audioset_tagging_cnn/")
#sys.path.append("/content/audioset_tagging_cnn/pytorch")
#from audioset_tagging_cnn.pytorch.models import ResNet22, ResNet38

In [ ]:
import numpy as np
import pandas as pd
import os
import tqdm
import random
import time

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import Adam, AdamW, SGD
from torchvision.models import resnet18, resnet50, densenet121
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

import matplotlib.pyplot as plt

from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

from contextlib import contextmanager
from typing import Optional
import logging
from numpy.random import beta
#import panns_inference
#from panns_inference import SoundEventDetection
#from panns_inference.pytorch_utils import interpolate, pad_framewise_output

device = torch.device('cuda')
#sed = SoundEventDetection(device='cuda')

In [ ]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

class config:
    SEED = 416
    N_FOLDS = 5
    FOLD = 0
    PRETRAINED = True
    TRAIN_INPUT = "./train_img_5"
    VALID_INPUT = "./train_img_5"
    INITIAL_EPOCH = 0
    OUTPUT = "./drive/My Drive/Study/Bird/output/from_resnet18_49"
    N_LABEL = 264
    
    TRAIN_BS = 256 #//4
    VALID_BS = 256 #//4
    TRAIN_WORKS = 0
    VALID_WORKS = 0
    
    DROPOUT_RATE = 0.2
    N_UNIT = 512 #*2
    EPOCHS = 55
    
    LR = 1e-3
    ALPHA = 0.2
    T_MAX = 10

    SMOOTH = 0.2

!mkdir -p "{config.OUTPUT}"

In [ ]:
"""train_df = pd.read_csv("./drive/My Drive/Study/Bird/birdsong-recognition/train.csv")
#weights = torch.tensor((1/train_df["ebird_code"].value_counts()).reset_index(drop=False).sort_values("index")["ebird_code"].values).to(device)

train_df["multi_label"] = train_df.apply(lambda x: [x["primary_label"]] + eval(x["secondary_labels"]) ,axis=1)

primary_label2ebird_code = {
    df["primary_label"].unique()[0]: ebird_code 
    for ebird_code, df in train_df[["ebird_code", "primary_label"]].groupby("ebird_code")
}

lst = []
for multi_label in train_df["multi_label"]:
    _lst = []
    for lab in multi_label:
        try:
            code = primary_label2ebird_code[lab]
        except KeyError:
            continue
        _lst.append(code)
    lst.append(_lst)
train_df["multi_ebird_code"] = lst

keys = set(train_df.ebird_code)
values = np.arange(0, len(keys))
code_dict = dict(zip(sorted(keys), values))


def multi_smooth_label(x):
    ebird_code = code_dict[x["ebird_code"]]
    multi_ebird_code = list(map(lambda x: code_dict[x], x["multi_ebird_code"]))
    t = torch.eye(config.N_LABEL)[ebird_code].numpy()
    multi_t = torch.eye(config.N_LABEL)[multi_ebird_code].sum(0).numpy()
    return (t * (1-config.SMOOTH) + multi_t * config.SMOOTH).tolist()

multi_targets = train_df[["ebird_code", "multi_ebird_code"]].apply(multi_smooth_label, axis=1).tolist()
multi_targets = list(map(str, multi_targets))"""

In [ ]:
class FreqMask:
    def __init__(self, F=30, num_masks=1, replace_with_zero=True):
        self.F = F
        self.num_masks = num_masks
        self.replace_with_zero = replace_with_zero

    def __call__(self, spec):
        cloned = spec.clone()
        num_mel_channels = cloned.shape[1]
    
        for i in range(0, self.num_masks):        
            f = random.randrange(0, self.F)
            f_zero = random.randrange(0, num_mel_channels - f)

            # avoids randrange error if values are equal and range is empty
            if (f_zero == f_zero + f): return cloned

            mask_end = random.randrange(f_zero, f_zero + f) 
            if (self.replace_with_zero): cloned[:, f_zero:mask_end] = 0
            else: cloned[:, f_zero:mask_end] = cloned.mean()
    
        return cloned

def get_dataloder():
    train_transform = transforms.Compose([
        transforms.RandomCrop((128-35, 313), pad_if_needed=True, padding_mode="constant"),
        #transforms.RandomCrop((64, 501), pad_if_needed=True, padding_mode="constant"),
        transforms.RandomApply([
            transforms.Lambda(lambda img: transforms.functional.adjust_gamma(img, gamma=2, gain=1)),
        ], p=0.5),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        transforms.RandomApply([
            FreqMask(replace_with_zero=False),
        ], p=0.5), 
    ])
    valid_transform = transforms.Compose([
        transforms.CenterCrop((128-35, 313)),
        #transforms.CenterCrop((64, 501)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

    train_datasets = datasets.ImageFolder(root=config.TRAIN_INPUT, transform=train_transform)
    valid_datasets = datasets.ImageFolder(root=config.VALID_INPUT, transform=valid_transform)

    #train_datasets.samples = [(s[0], t)for s, t in zip(train_datasets.samples, multi_targets)]

    skf = StratifiedKFold(n_splits=config.N_FOLDS, shuffle=True, random_state=config.SEED)
    _t = train_datasets.targets
    trn_idx, val_idx = [(trn_idx, val_idx) for trn_idx, val_idx in skf.split(_t, _t)][config.FOLD]

    train_datasets = torch.utils.data.Subset(train_datasets, trn_idx)
    valid_datasets = torch.utils.data.Subset(valid_datasets, val_idx)

    train_data_loader = torch.utils.data.DataLoader(train_datasets, batch_size=config.TRAIN_BS, shuffle=True, num_workers=config.TRAIN_WORKS)
    valid_data_loader = torch.utils.data.DataLoader(valid_datasets, batch_size=config.VALID_BS, shuffle=False, num_workers=config.VALID_WORKS)
    
    return train_data_loader, valid_data_loader

data_loader, _ = get_dataloder()
#_, data_loader = get_dataloder()
for d in data_loader:
    break
img = d[0][0]
plt.imshow(np.rollaxis(img.numpy(), 0, 3))

In [ ]:
def loss_fn(output, target):
    #loss = nn.BCEWithLogitsLoss(pos_weight=weights)(output, target)
    loss = nn.BCEWithLogitsLoss()(output, target)
    return loss

def mixup(input, target, gamma):
    # target is onehot format!
    perm = torch.randperm(input.size(0))
    perm_input = input[perm]
    perm_target = target[perm]
    return input.mul_(gamma).add_(1 - gamma, perm_input), target.mul_(gamma).add_(1 - gamma, perm_target)


class BirdcallNet(nn.Module):
    def __init__(self):
        super(BirdcallNet, self).__init__()
        resnet = resnet18(pretrained=config.PRETRAINED)
        self.resnet_head = nn.Sequential(*list(resnet.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=config.DROPOUT_RATE)
        self.fc = nn.Linear(config.N_UNIT, config.N_LABEL)
        #self.fc = AttBlock(config.N_UNIT, config.N_LABEL, activation="linear")

        #self.densenet = densenet121(pretrained=config.PRETRAINED)
        #self.densenet.classifier = nn.Linear(config.N_UNIT, config.N_LABEL)

        #self.sed = nn.Sequential(*list(sed.model.module.children())[4:-1])
        #self.ave_pool = nn.AdaptiveAvgPool2d((1, 1))
        #self.max_pool = nn.AdaptiveMaxPool2d((1, 1))
        #self.fc = AttBlock(2048, config.N_LABEL)


    def forward(self, x):
        #x = x.transpose(3, 2)
        h = self.resnet_head(x)
        h = self.pool(h)
        h = h.view(-1, config.N_UNIT)
        #h = torch.mean(h, dim=3)

        #x1 = F.max_pool1d(h, kernel_size=3, stride=1, padding=1)
        #x2 = F.avg_pool1d(h, kernel_size=3, stride=1, padding=1)
        #h = x1 + x2

        h = self.dropout(h)
        logits = self.fc(h)
        return {"clipwise_output": logits}
        #clipwise_output, norm_att, segmentwise_output = self.fc(h)
        #segmentwise_output = segmentwise_output.transpose(1, 2)
        #framewise_output = interpolate(segmentwise_output, 31)
        #framewise_output = pad_framewise_output(framewise_output, 313)
        #return clipwise_output

        #return self.densenet(x)

In [ ]:
def train_fn(train_data_loader, model, optimizer, scheduler=None):
    losses, lrs = [], []
    model.train()
    t = tqdm.notebook.tqdm(train_data_loader, total=len(train_data_loader))
    for (X, y) in t:
        y_onehot = torch.eye(config.N_LABEL)[y]
        #y_onehot = torch.eye(config.N_LABEL+1)[y][:,1:]
        #y_onehot = torch.tensor(list(map(eval, y)))

        _X, _y = mixup(X, y_onehot, beta(config.ALPHA, config.ALPHA))

        #y_pred = model(_X.to(device))
        #y_pred = model(_X.to(device)[:,0,:,:].unsqueeze(1))
        output = model(_X.to(device))
        y_pred = output["clipwise_output"]

        loss = loss_fn(y_pred,  _y.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()

        losses.append(loss.item())
        lrs.append(np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean())
    
    return sum(losses)/len(losses), lrs


def get_single_label_from_multi_predict(y, y_pred, threshould):
    lst = []
    for idx in range(len(y_pred)):
        p = y_pred[idx]
        if sum(p >= threshould) < 2:
            _p = p.argmax().numpy()
        else:
            _p = np.where(p >= threshould)
            _p = _p[0]
            if y[idx].numpy() in _p:
                _p = y[idx].numpy()
            else:
                _p = p.argmax().numpy()
        lst.append(_p)
    return np.array(lst)
        
def valid_fn(valid_data_loader, model, threshould=0.5):
    losses, f1_lst_a, f1_lst_b = [], [], []
    model.eval()
    t = tqdm.notebook.tqdm(valid_data_loader, total=len(valid_data_loader))
    for (X, y) in t:

        lst = []
        with torch.no_grad():
            #y_pred = model(X.to(device))
            #y_pred = model(X.to(device)[:,0,:,:].unsqueeze(1))
            output = model(X.to(device))
            y_pred = output["clipwise_output"]

        _y = torch.eye(config.N_LABEL)[y]
        #_y = torch.eye(config.N_LABEL+1)[y][:,1:]

        loss = loss_fn(y_pred,  _y.to(device))
        #loss = nn.BCEWithLogitsLoss()(y_pred,  _y.to(device))
        losses.append(loss.item())

        y_pred_a = get_single_label_from_multi_predict(y, y_pred.sigmoid().cpu(), threshould)
        #y_pred_a = get_single_label_from_multi_predict(y-1, y_pred.sigmoid().cpu(), threshould)
        y_pred_b = y_pred.argmax(1).cpu()

        f1_a = f1_score(y, y_pred_a, average="micro")
        f1_b = f1_score(y, y_pred_b, average="micro")
        #f1_a = f1_score(y-1, y_pred_a, average="micro")
        #f1_b = f1_score(y-1, y_pred_b, average="micro")
        f1_lst_a.append(f1_a)
        f1_lst_b.append(f1_b)

    return sum(f1_lst_a)/len(f1_lst_a), sum(f1_lst_b)/len(f1_lst_b), sum(losses)/len(losses)

In [ ]:
print(f"### Fold-{config.FOLD} ###")

set_seed(config.SEED+config.FOLD)

train_data_loader, valid_data_loader = get_dataloder()

model = BirdcallNet()
model.to(device)

optimizer = Adam(model.parameters(), lr=config.LR)
#optimizer = SGD(model.parameters(), lr=config.LR)
scheduler = CosineAnnealingLR(optimizer, T_max=len(train_data_loader)*config.T_MAX, eta_min=0.0)

if config.INITIAL_EPOCH == 0:
    best_loss, best_score_a, best_score_b = 9999, 0, 0
    trn_losses, trn_lrs, val_losses, val_scores_a, val_scores_b = [], [], [], [], []
else:
    model.load_state_dict(torch.load(f"{config.OUTPUT}/birdcallnet_f{config.FOLD}_latest_model.bin"))
    optimizer.load_state_dict(torch.load(f"{config.OUTPUT}/birdcallnet_f{config.FOLD}_latest_optimizer.bin"))
    scheduler.load_state_dict(torch.load(f"{config.OUTPUT}/birdcallnet_f{config.FOLD}_latest_scheduler.bin"))
    log_df = pd.read_csv(f"{config.OUTPUT}/valid_f1_fold{config.FOLD}_score.csv")
    _, trn_losses, val_losses, val_scores_a, val_scores_b = log_df.values.T.tolist()
    best_loss, best_score_a, best_score_b = min(val_losses), max(val_scores_a), max(val_scores_b)
    trn_lrs = []


for epoch in range(config.INITIAL_EPOCH, config.EPOCHS):
    print(f"{epoch} epoch")
    tloss, lrs = train_fn(train_data_loader, model, optimizer, scheduler)
    val_f1_a, val_f1_b, vloss = valid_fn(valid_data_loader, model)

    # save best score model
    if best_score_a <= val_f1_a:
        best_score_a = val_f1_a
        torch.save(model.state_dict(), f"{config.OUTPUT}/birdcallnet_f{config.FOLD}_best_score_a.bin")
        print(f"Best Score A Update!!! -> {best_score_a}")

    # save best score model
    if best_score_b <= val_f1_b:
        best_score_b = val_f1_b
        torch.save(model.state_dict(), f"{config.OUTPUT}/birdcallnet_f{config.FOLD}_best_score_b.bin")
        print(f"Best Score B Update!!! -> {best_score_b}")

    # save best loss model
    if best_loss >= vloss:
        best_loss = vloss
        torch.save(model.state_dict(), f"{config.OUTPUT}/birdcallnet_f{config.FOLD}_best_loss.bin")
        print(f"Best Loss Update!!! -> {best_loss}")

    torch.save(model.state_dict(), f"{config.OUTPUT}/birdcallnet_f{config.FOLD}_latest_model.bin")
    torch.save(optimizer.state_dict(), f"{config.OUTPUT}/birdcallnet_f{config.FOLD}_latest_optimizer.bin")
    torch.save(scheduler.state_dict(), f"{config.OUTPUT}/birdcallnet_f{config.FOLD}_latest_scheduler.bin")

    # save training logs
    trn_losses.append(tloss)
    val_losses.append(vloss)
    val_scores_a.append(val_f1_a)
    val_scores_b.append(val_f1_b)
    trn_lrs.extend(lrs)
    log_df = pd.DataFrame(zip(trn_losses, val_losses, val_scores_a, val_scores_b), columns=["train loss", "valid loss", "score_a", "score_b"])
    log_df.to_csv(f"{config.OUTPUT}/valid_f1_fold{config.FOLD}_score.csv", index=True)

In [ ]:
print(f"Best Score　A: {best_score_a} / Best Score　B: {best_score_b} / Best Loss: {best_loss}")
plt.plot(trn_lrs); plt.show()
plt.plot(val_scores_a); plt.show()
plt.plot(val_scores_b); plt.show()
plt.plot(trn_losses)
plt.plot(val_losses)
plt.show()